## Change Runtime Type to T4 GPU in Google Colab


Following code is adapted from the  book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition)
(chapter 11: Deep learning for text).
This example, is a sequence-to-sequence Transformer model, which
we'll train on an Igbo-to-English machine translation task.

WHat we do:

- Vectorize text using the Keras `TextVectorization` layer.
- Implement a `TransformerEncoder` layer, a `TransformerDecoder` layer,
and a `PositionalEmbedding` layer.
- Prepare data for training a sequence-to-sequence model.
- Use the trained model to generate translations of never-seen-before
input sentences (sequence-to-sequence inference).


* use the declaration_top_10.txt file to test run code



In [2]:
# install version 2.8 as 2.15 has changed few features
!pip install tensorflow-gpu==2.8.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 51.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing 

In [1]:
import tensorflow as tf

print(tf.__version__)

2.8.0


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
import re
import pathlib
import random
import string


#read our sample data from UDHR
with open("declartation_top_10.txt") as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    igbo, eng = line.split("\t")
    eng = "[start] " + eng + " [end]"
    text_pairs.append((igbo, eng))




random.seed(42)
random.shuffle(text_pairs)
num_val_samples = int(0.40 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

#print(f"{len(text_pairs)} total pairs")
#print(f"{len(train_pairs)} training pairs")
#print(f"{len(val_pairs)} validation pairs")
#print(f"{len(test_pairs)} test pairs")

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


igbo_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)
eng_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

train_igbo_texts = [pair[0] for pair in train_pairs]
train_eng_texts = [pair[1] for pair in train_pairs]

igbo_vectorization.adapt(train_igbo_texts)
eng_vectorization.adapt(train_eng_texts)

def format_dataset(igbo, eng):
    igbo = igbo_vectorization(igbo)
    eng = eng_vectorization(eng)
    return ({"encoder_inputs": igbo, "decoder_inputs": eng[:, :-1],}, eng[:, 1:])


def make_dataset(pairs):
    igbo_texts, eng_texts = zip(*pairs)
    igbo_texts = list(igbo_texts)
    eng_texts = list(eng_texts)
    dataset = tf.data.Dataset.from_tensor_slices((igbo_texts, eng_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")


from enum import Enum
class Mode(Enum):
    TRAINING = 1
    TESTING = 2
    VALIDATION = 3

# Change the MODE as needed
MODE = Mode(1)

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config["embed_dim"] = self.embed_dim
        config["dense_dim"] = self.dense_dim
        config["num_heads"] = self.num_heads
        config["attention"] = self.attention
        config["dense_proj"] = self.dense_proj
        config["layernorm_1"] = self.layernorm_1
        config["layernorm_2"] = self.layernorm_2
        config["supports_masking"] = self.supports_masking
        return config

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def get_config(self):
        config = super().get_config()
        config["embed_dim"] = self.embed_dim
        config["token_embeddings"] = self.token_embeddings
        config["position_embeddings"] = self.position_embeddings
        config["vocab_size"] = self.vocab_size
        config["sequence_length"] = self.sequence_length
        return config

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True


    def get_config(self):
        config = super().get_config()
        config["embed_dim"] = self.embed_dim
        config["latent_dim"] = self.latent_dim
        config["num_heads"] = self.num_heads
        config["attention_1"] = self.attention_1
        config["attention_2"] = self.attention_2
        config["dense_proj"] = self.dense_proj
        config["layernorm_1"] = self.layernorm_1
        config["layernorm_2"] = self.layernorm_2
        config["layernorm_3"] = self.layernorm_3
        config["supports_masking"] = self.supports_masking
        return config

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

embed_dim = 512
latent_dim = 4096
num_heads = 16

if MODE == Mode.TRAINING or MODE == Mode.VALIDATION:
    encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
    x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
    encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
    encoder = keras.Model(encoder_inputs, encoder_outputs)

    decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
    encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
    x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
    x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
    x = layers.Dropout(0.5)(x)
    decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
    decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

    decoder_outputs = decoder([decoder_inputs, encoder_outputs])
    transformer = keras.Model(
        [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
    )

transformer.summary()


TENSORBOARD_DIR = pathlib.Path("./runs")
CHECKPOINT_DIR = pathlib.Path("./chkp")
TENSORBOARD_DIR.mkdir(exist_ok=True)
CHECKPOINT_DIR.mkdir(exist_ok=True)

checkpoint_filepath = pathlib.Path(CHECKPOINT_DIR, "weights-ep:{epoch}-acc:{val_accuracy:.3f}.ckpt")

if MODE == Mode.TRAINING:
    checkpoints_callback = keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        save_freq='epoch',
        save_weights_only=True,
        verbose=1,
    )
    tensorboard_callback = keras.callbacks.TensorBoard(
        log_dir=TENSORBOARD_DIR,
        histogram_freq=1)

    early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=4,
        mode="auto",
        restore_best_weights=True,
    )
    lr_callback = keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.1,
        patience=2,
        mode="auto",
    )


epochs =25 #40

# transformer.summary()
# transformer.summary()
if MODE == Mode.TRAINING:
    transformer.compile("adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    transformer.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[tensorboard_callback, checkpoints_callback, early_stopping, lr_callback])


final_model = pathlib.Path("final_model")
final_model.mkdir(exist_ok=True)

if MODE == Mode.TRAINING:
    transformer.save(str(final_model))

if MODE == Mode.VALIDATION:
    latest = tf.train.latest_checkpoint("../input/nlp-machine-translation/chkp")
    transformer.load_weights(latest)
elif MODE == Mode.TESTING:
    transformer = tf.saved_model.load("../input/nlp-machine-translation/final_model")


eng_vocab = eng_vectorization.get_vocabulary()
eng_index_lookup = dict(zip(range(len(eng_vocab)), eng_vocab))
max_decoded_sentence_length = 20

import itertools
dict(itertools.islice(eng_index_lookup.items(), 9)) # eng_index_lookup is dictionary


def decode_sequence(input_sentence):
    tokenised_input_sentence = igbo_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenised_target_sentence = eng_vectorization([decoded_sentence])[:,:-1]
        predictions = transformer([tokenised_input_sentence, tokenised_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = eng_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence

for _ in range(10):
    sentence = random.choice(test_pairs)
    translated = decode_sequence(sentence[0])
    print("-"*40)
    print("igbo input:", sentence[0])
    print("Target sentence;", sentence[1])
    print("Translated sentence:", translated)


input_sentence = "Onye o̟ bu̟la nwere ikike inwe ndu̟, ohere na nchedo nke onwe ya."
decode_sequence(input_sentence)

inputs["encoder_inputs"].shape: (9, 20)
inputs["decoder_inputs"].shape: (9, 20)
targets.shape: (9, 20)
Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding_6 (Positi  (None, None, 512)   7690240     ['encoder_inputs[0][0]']         
 onalEmbedding)                                                                                   
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                    

----------------------------------------
igbo input: Onye o̟ bu̟la nwere ikike inwe ndu̟, ohere na nchedo nke onwe ya.
Target sentence; [start] Everyone has the right to life, liberty and security of person. [end]
Translated sentence: [start] everyone the shall be to to and to in the and to [end]
----------------------------------------
igbo input: Onye o̟ bu̟la nwere ikike inwe ndu̟, ohere na nchedo nke onwe ya.
Target sentence; [start] Everyone has the right to life, liberty and security of person. [end]
Translated sentence: [start] everyone the shall be to to and to in the and to [end]
----------------------------------------
igbo input: Nkeji edemede 13.
Target sentence; [start] Article 13. [end]
Translated sentence: [start] article one shall be to to [end]
----------------------------------------
igbo input: Onye o̟ bu̟la nwere ikike i̟bu̟ nwa afo̟ n'ala ya.
Target sentence; [start] Everyone has the right to a nationality. [end]
Translated sentence: [start] everyone the shall be t

'[start] everyone the shall be to to and to in the and to [end]'